In [1]:
import os
import sys
from dotenv import load_dotenv
from pathlib import Path

if sys.platform == 'linux':
    load_dotenv(dotenv_path=Path('.') / '.env.linux')
elif sys.platform == 'win32':
    load_dotenv(dotenv_path=Path('.') / '.env.win')
else:
    raise ValueError('Ваша операционная система не поддерживается')

os.environ['HF_HOME'] = os.getenv('HUGGING_FACE_CACHE_DIR')
DATASET_PATH = os.getenv('DATASET_PATH', None)
print(DATASET_PATH)
DATASET_FOLDER = os.getenv('DATASET_FOLDER_PATH', None)
MODEL_NAME = os.getenv('MODEL_NAME', None)
MODEL_SAVE_DIR=os.getenv('MODEL_SAVE_DIR', None)
MODEL_SAVE_NAME = os.getenv('MODEL_SAVE_NAME', None)
print(MODEL_NAME)

/mnt/12A4CA9DA4CA8329/Files/Datasets/recipes_generation/fine_tuning_preprocessed.csv
Qwen/Qwen3-4B-Instruct-2507


In [2]:
from peft import PeftConfig, PeftModel, LoraConfig, get_peft_model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from datasets import load_from_disk, load_dataset

Рекомендации по выбору lora_alpha.\
h = Wx + (α/r) * BA * x -- формула расчета активаций.

r = 8, lora_alpha = 16  # alpha = 2*r (агрессивнее обучение, из оригинальной статьи)\
r = 8, lora_alpha = 8   # alpha = r (баланс)\
r = 8, lora_alpha = 4   # alpha = r/2 (более медленное, но стабильное обучение)

In [3]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "o_proj", "k_proj", "up_proj", "down_proj", "gate_proj"],
    bias="none",
    task_type="CAUSAL_LM")

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.bfloat16,
    trust_remote_code=True)

model = get_peft_model(base_model, peft_config)

# Количество обучаемых параметров
model.print_trainable_parameters()
# model.config.use_cache = False
# model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 66,060,288 || all params: 4,088,528,384 || trainable%: 1.6157


Лучше, перед обучением заморозить все веса основной модели в цикле

In [5]:
full_dataset = load_dataset('csv', data_files=[DATASET_PATH])
full_dataset = full_dataset.remove_columns(['composition_list', 'prompt'])

In [6]:
def get_assistant_content(row):
    return {'assistant_content' :\
f'''Ингридиенты:
{row['ingredients']}

Процесс приготовления:
{row['instructions']}'''}

full_dataset = full_dataset.map(get_assistant_content, batched=False)

In [7]:
# Preprocesses the dataset
def preprocess_dataset_with_eos(eos_token, dataset):
    def preprocess_function(examples):
        instructions = examples["assistant_content"]
        names = examples["name"]

        chats = []
        for recipe, name in zip(instructions, names):
            # Append the EOS token to the response
            recipe += eos_token

            chat = [
                {"role": "user", "content": f"Как я могу приготовить {name}?"},
                {"role": "assistant", "content": recipe},
            ]

            chats.append(chat)
        return {"messages": chats}
    dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
    return dataset

In [8]:
EOS_TOKEN = tokenizer.eos_token
dataset = full_dataset['train'].train_test_split(test_size=0.1)
dataset = preprocess_dataset_with_eos(EOS_TOKEN, dataset)

Map:   0%|          | 0/25095 [00:00<?, ? examples/s]

Map:   0%|          | 0/2789 [00:00<?, ? examples/s]

In [9]:
def formatting_function(examples):
    return { 'chat' : tokenizer.apply_chat_template(
        examples["messages"],
        tokenize=False,
        add_generation_prompt=False)}

dataset = dataset.map(formatting_function, batched=True).remove_columns('messages')

Map:   0%|          | 0/25095 [00:00<?, ? examples/s]

Map:   0%|          | 0/2789 [00:00<?, ? examples/s]

In [10]:
dataset['train'][0]

{'chat': '<|im_start|>user\nКак я могу приготовить Круаcсаны с\xa0яблоками?<|im_end|>\n<|im_start|>assistant\nИнгридиенты:\n Дрожжевое слоеное тесто 500.0 гр.\n Сахар 1.0 стак.\n Яблоко 2.0 шт.\n Сливочное масло 300.0 гр.\n Ванилин по вкусу по_вкусу.\n Корица по вкусу по_вкусу.\n\nПроцесс приготовления:\n1. Яблоки почистить, нарезать ломтиками.\n2. Каждую пластину теста разрезать по диагонали, положить ломтик яблока.\n3. Свернуть рулетом.\n4. Масло растопить с сахаром, добавить ванилин.\n5. Залить им слойки.\n6. Посыпать корицей.\n7. Поставить в духовку при средней температуре на 25–30 минут.\n8. Перед подачей на стол дать остыть, а также можно присыпать сахарной пудрой.<|im_end|><|im_end|>\n'}

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['chat'],
        num_rows: 25095
    })
    test: Dataset({
        features: ['chat'],
        num_rows: 2789
    })
})

In [12]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # Отключаем masked language modelling (mlm)

In [13]:
tokenized_dataset = dataset.map(lambda x: tokenizer(x['chat'], padding=True, truncation=True, max_length=680), batched=True).remove_columns('chat')

Map:   0%|          | 0/25095 [00:00<?, ? examples/s]

Map:   0%|          | 0/2789 [00:00<?, ? examples/s]

In [14]:
args = TrainingArguments(
    output_dir=os.path.join(MODEL_SAVE_DIR, 'training_checkpoints'),  # Кастомная директория
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.01,
    warmup_steps=200,
    lr_scheduler_type="linear",
    learning_rate=5e-5,
    save_steps=700,
    eval_steps=700,
    logging_steps=700,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,  # Загружать лучшую модель
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=torch.cuda.is_available(),  # Только если GPU поддерживает
    remove_unused_columns=False,
    gradient_checkpointing=False,  # Экономия памяти
    save_total_limit=2,  # Хранить только 2 лучших чекпоинта
    push_to_hub=False,
    report_to="none",  # Отключить логирование в сторонние сервисы
)

trainer = Trainer(
    model=model,
    processing_class=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

In [15]:
# Обучение
print("Начало обучения...")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Начало обучения...


Step,Training Loss,Validation Loss
700,1.099300,0.947536
1400,0.953800,0.914579
2100,0.931300,0.897632
2800,0.912000,0.887105


TrainOutput(global_step=3137, training_loss=0.9675343847593989, metrics={'train_runtime': 15107.2865, 'train_samples_per_second': 1.661, 'train_steps_per_second': 0.208, 'total_flos': 3.787903210420224e+17, 'train_loss': 0.9675343847593989, 'epoch': 1.0})

Переделать сохранение модели!!!

In [16]:
# Сохраняем только адаптеры
model.save_pretrained(os.path.join(MODEL_SAVE_DIR, MODEL_SAVE_NAME))
tokenizer.save_pretrained(os.path.join(MODEL_SAVE_DIR, MODEL_SAVE_NAME))

print(f'Модель сохранена в {os.path.join(MODEL_SAVE_DIR, MODEL_SAVE_NAME)}')

Модель сохранена в /mnt/2FADF63B267AA05B/AI_models/recepies_generation/Qwen/Qwen3-4B-Instruct-2507-recepies


In [17]:
dfdf

NameError: name 'dfdf' is not defined

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

peft_config = PeftConfig.from_pretrained(os.path.join(MODEL_SAVE_DIR, MODEL_SAVE_NAME))

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.bfloat16,
    trust_remote_code=True)

model = PeftModel.from_pretrained(base_model, model_id=os.path.join(MODEL_SAVE_DIR, MODEL_SAVE_NAME), is_trainable=True)

# Количество обучаемых параметров
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 66,060,288 || all params: 4,088,528,384 || trainable%: 1.6157


In [22]:
dish_name = 'спички'
chat = [{'role' : 'system' , 'content' : 'Ты - интеллектуальный помощник для приготовления пищи. Твоя задача - давать пользователю подробный рецепт приготовления блюда',\
         'role' : 'user', 'content' : f'Я хотел бы приготовить {dish_name}. Напиши рецепт со всеми подробностями.'}]
message = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt = True)

In [23]:
message

'<|im_start|>user\nЯ хотел бы приготовить спички. Напиши рецепт со всеми подробностями.<|im_end|>\n<|im_start|>assistant\n'

In [26]:
from transformers import TextStreamer

model.to('cuda').eval()

output = model.generate(
    **tokenizer(message, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024,
    temperature = 0.7, top_p = 0.8, top_k = 20, # recomended for non thinking
    streamer = TextStreamer(tokenizer),)

<|im_start|>user
Я хотел бы приготовить спички. Напиши рецепт со всеми подробностями.<|im_end|>
<|im_start|>assistant
Ингридиенты:
 Яйцо куриное 0.5 шт.
 Сахар 0.1 стак.
 Пшеничная мука 0.2 стак.
 Соль щепотка по_вкусу.
 Разрыхлитель 0.1 чайн.
 Ванильный сахар 0.1 чайн.
 Сливочное масло 12.5 гр.
 Сода 0.1 чайн.
 Молоко 0.1 стак.
 Сливки 35%-ные 0.1 стак.
 Ванильный сахар по вкусу по_вкусу.
 Сахарная пудра по вкусу по_вкусу.

Процесс приготовления:
1. Разогрейте духовку до 180 градусов.
2. В большой миске смешайте яйца, сахар, муку, соль, разрыхлитель, ванильный сахар, соду и сливочное масло. Перемешайте миксером на средней скорости до однородной массы.
3. Добавьте молоко и сливки и взбейте миксером на высокой скорости до однородной массы. Затем введите 1 столовую ложку ванильного сахара.
4. Разложите тесто по формочкам для спичек. Выпекайте 10 минут, затем уменьшите температуру до 150 градусов и выпекайте еще 20 минут. Выньте спички из формочек и охладите.
5. Для крема смешайте сахарну